In [23]:


# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")
# Pkg.add("Formatting")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
using Formatting
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")



PyObject <module 'cv2' from '/usr/lib/python3/dist-packages/cv2.cpython-38-x86_64-linux-gnu.so'>

In [40]:
function Laplacian_3D!(P₁,P₂,B₁,B₂,Δx)
    A₁ = CircularArray(B₁);
    A₂ = CircularArray(B₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j,k) = Tuple(😄)
        P₁[i,j,k] = @fastmath (A₁[i+1,j,k] + A₁[i-1,j,k] + A₁[i,j + 1,k] + A₁[i,j - 1,k] + A₁[i,j,k + 1] + A₁[i,j,k-1] - 6A₁[i,j,k])/(Δx^2);
        P₂[i,j,k] = @fastmath (A₂[i+1,j,k] + A₂[i-1,j,k] + A₂[i,j + 1,k] + A₂[i,j - 1,k] + A₂[i,j,k + 1] + A₂[i,j,k-1] - 6A₂[i,j,k])/(Δx^2);
    end
    return nothing
end


function Make_Oscillon!(angle,A,N,σ)
    for i ∈ 1:N
        for j ∈ 1:N
            for k ∈ 1:N
                x = abs(i - (N+1)/2)
                y = abs(j - (N+1)/2)
                z = abs(k = (N+1)/2)
                angle[i,j,k] = A/cosh(sqrt(x^2 + y^2 + z^2)/σ)
            end
        end
    end
    return nothing
end

function Make_WOscillon!(angle,A,N,σ)
    B = rand(Normal(1,1e-1),N,N)
    for i ∈ 1:N
        for j ∈ 1:N
            for k ∈ 1:N
                x = abs(i - (N+1)/2)
                y = abs(j - (N+1)/2)
                z = abs(k - (N+1)/2)
            
                angle[i,j,k] = A/cosh(sqrt(x^2 + y^2 + z^2)/σ) * B[i,j,k]
            end
        end
    end
    return nothing
end


function Anti_string!(A₁,A₂,N,Σ,varx,vary,varz)
    for i ∈ 1:N 
        for j ∈ 1:N      
            for k ∈ 1:N
                xclose = i - varx
                yclose = vary - j
                xfar = i - (N+1)/2
                yfar = (N+1)/2 - j
                t = exp(-20(xclose^2 + yclose^2)/N^2)
                x = xclose*t + (1-t)*xfar
                y = yclose*t + (1-t)*yfar

                θ = angle(x + y*im)
                A₁[i,j,k] = Σ*cos(θ)
                A₂[i,j,k] = Σ*sin(θ)
            end
        end
    end
    return nothing
end


function Clock_string!(A₁,A₂,N,Σ,varx,vary,varz)
    for i ∈ 1:N 
        for j ∈ 1:N 
            for k ∈ 1:N
                xclose = i - varx
                yclose = vary - j
                xfar = i - (N+1)/2
                yfar = (N+1)/2 - j
                t = exp(-20(xclose^2 + yclose^2)/N^2)
                x = xclose*t + (1-t)*xfar
                y = yclose*t + (1-t)*yfar

                θ = angle(x - y*im)
                A₁[i,j,k] = Σ*cos(θ) 
                A₂[i,j,k] = Σ*sin(θ) 
            end
        end
    end
    return nothing
end

function slot!(A,B,a,b,c)
    A = CircularArrays.CircularArray(A)
    M = length(B[:,:,1])
    N = length(B[:,1,:])
    O = length(B[1,:,:])
    for i ∈ 1:M
        for j ∈ 1:N
            for k ∈ 1:O
                A[Int(a)+i-Int(M/2),Int(b)+j-Int(N/2) ,Int(c)+k - Int(O/2)] = B[i,j,k]
            end
        end
    end
    return nothing
end



function Cores_3D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            for 🎅 ∈ 1:(N-1)
                anorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                anorm₂ = (angle[😄+1,🥪,🎅] + π)/(2π)
                anorm₃ = (angle[😄+1,🥪+1,🎅] + π)/(2π)
                anorm₄ = (angle[😄,🥪+1,🎅] + π)/(2π)

                bnorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                bnorm₂ = (angle[😄+1,🥪,🎅] + π)/(2π)
                bnorm₃ = (angle[😄+1,🥪,🎅+1] + π)/(2π)
                bnorm₄ = (angle[😄,🥪,🎅+1] + π)/(2π)

                cnorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                cnorm₂ = (angle[😄,🥪+1,🎅] + π)/(2π)
                cnorm₃ = (angle[😄,🥪+1,🎅+1] + π)/(2π)
                cnorm₄ = (angle[😄,🥪,🎅+1] + π)/(2π)

                aθ₁ = min(abs(anorm₂ - anorm₁),1-abs(anorm₂ - anorm₁))
                aθ₂ = min(abs(anorm₃ - anorm₂),1-abs(anorm₃ - anorm₂))
                aθ₃ = min(abs(anorm₄ - anorm₃),1-abs(anorm₄ - anorm₃))

                bθ₁ = min(abs(bnorm₂ - bnorm₁),1-abs(bnorm₂ - bnorm₁))
                bθ₂ = min(abs(bnorm₃ - bnorm₂),1-abs(bnorm₃ - bnorm₂))
                bθ₃ = min(abs(bnorm₄ - bnorm₃),1-abs(bnorm₄ - bnorm₃))

                cθ₁ = min(abs(cnorm₂ - cnorm₁),1-abs(cnorm₂ - cnorm₁))
                cθ₂ = min(abs(cnorm₃ - cnorm₂),1-abs(cnorm₃ - cnorm₂))
                cθ₃ = min(abs(cnorm₄ - cnorm₃),1-abs(cnorm₄ - cnorm₃))

                aθ = aθ₁ + aθ₂ + aθ₃
                bθ = bθ₁ + bθ₂ + bθ₃
                cθ = cθ₁ + cθ₂ + cθ₃
                θₛ = aθ + bθ + cθ

                if θₛ > accept 
                    append!(s,[[😄,🥪,🎅]])
                end
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]
            diffₜ = s[🇸🇦 + 1][3] - s[🇸🇦][3]

            if diffᵣ == 0 && diffₛ == 1 && diffₜ == 0
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0 && diffₜ == 0
                count += 1
            end
            if diffᵣ == 0 && diffₛ == 0 && diffₜ == 1
                count += 1
            end
        end
    end

    num = length(s) - count
    num = 1.5num

    return num,s
end




function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end


function setting_3D!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] < 5
            moo[t] = NaN
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j,k) = Tuple(😄)
        @fastmath moo[i,j,k] = sqrt(A₁[i,j,k]^2 + A₂[i,j,k]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j,k) = Tuple(😄)
        angle[i,j,k] = @fastmath atan(A₂[i,j,k],A₁[i,j,k]);
    end
    return nothing
end

function meshgrid(xin,yin,zin)
    nx=length(xin)
    ny=length(yin)
    nz=length(zin)
    xout=zeros(nz,ny,nx)
    yout=zeros(nz,ny,nx)
    zout=zeros(nz,ny,nx)
    for jx=1:nx
        for ix=1:ny
            for hx=1:nz
                xout[hx,ix,jx]=xin[jx]
                yout[hx,ix,jx]=yin[ix]
                zout[hx,ix,jx]=zin[hx]
            end
        end
    end
    return (x=xout, y=yout, z=zout)
end

function minmax(A)
    lower = minimum(minimum.(minimum.(A)))
    upper = maximum(maximum.(maximum.(A)))
    return lower,upper
end

minmax (generic function with 1 method)

In [25]:
function FPQrun_3D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats,cont,low)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N,N);
    M₂ = zeros(N,N,N);

    F₁ = zeros(N,N,N);
    F₂ = zeros(N,N,N);

    moo = zeros(N,N,N);
    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []

    axenergy = []
    
    lo = low
    
    tim = t₀
    
    for 🥪 ∈ 1+cont:repeats+cont
        
        


        while tim < 100*🥪 + t₀
            tim = round(tim,digits = 4);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng,loc = Cores_3D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)
                
                loc₁ = []
                loc₂ = []
                loc₃ = []

                for i ∈ 1:length(loc)
                    append!(loc₁,loc[i][1])
                    append!(loc₂,loc[i][2])
                    append!(loc₃,loc[i][3])
                end

                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(moo)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = 4/3 * π* Abin.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(angle)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = 4/3 * π* Abins.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])




                f_image = FFTW.fft( ( (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2 .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = 4/3 * π* Abint.* (kbins[2:end].^3 - kbins[1:end-1].^3)


                append!(axenergy,[Abint])

                ploot = Plots.plot(loc₁,loc₂,loc₃,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false,title = "η = "*string(tim))
                Plots.savefig(ploot,"3D/PQTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png")
                
            end
            
            if tim < 250
                PQupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            else
                PQupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            end
            lo += 1
        end
     


        save("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)




        save("3D/PQTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("3D/PQTOWN/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



    end



    return nothing
end

FPQrun_3D! (generic function with 1 method)

In [26]:
function EQCDfupdate_3D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j,k) = Tuple(😄)
        F₁[i,j,k] = @fastmath M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j,k]^2/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₁[i,j,k];
        F₂[i,j,k] = @fastmath M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j,k]*C₂[i,j,k]/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₂[i,j,k];
    end
    return nothing
end


function EQCDAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j,k) = Tuple(😄)
        A₁[i,j,k] = @fastmath A₁[i,j,k] .+ Δt .* (Ȧ₁[i,j,k] .+ 0.5Δt .* F₁[i,j,k])
        A₂[i,j,k] = @fastmath A₂[i,j,k] .+ Δt .* (Ȧ₂[i,j,k] .+ 0.5Δt .* F₂[i,j,k])
    end
    return nothing
end


function EQCDvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j,k) = Tuple(😄)
        Ȧ₁[i,j,k] = @fastmath Ȧ₁[i,j,k] .+ 0.5Δt .* (F₁[i,j,k] .+ M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j,k]^2/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₁[i,j,k])
        Ȧ₂[i,j,k] = @fastmath Ȧ₂[i,j,k] .+ 0.5Δt .* (F₂[i,j,k] .+ M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j,k]*C₂[i,j,k]/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₂[i,j,k])
    end
    return nothing
end


function EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s)

    n = 1

    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = [2.8 3 3.2 3.4 3.6 0 6 1]
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_3D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);

    EQCDvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)

    return nothing
end

EQCDupdate_3D! (generic function with 1 method)

In [27]:
function LLaplacian_3D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j,k) = Tuple(😄)
        P[i,j,k] = @fastmath (B[i+1,j,k] + B[i-1,j,k] + B[i,j + 1,k] + B[i,j - 1,k] + B[i,j,k + 1] + B[i,j,k-1] - 6B[i,j,k])/(Δx^2);
    end
    return nothing
end


function Lfupdate_3D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j,k) = Tuple(😄)
        F[i,j,k] = @fastmath M[i,j,k] - ηₓ^n * η^2 * sin(A[i,j,k]) - 2/η * Ȧ[i,j,k];
    end
    return nothing
end


function LAupdate_3D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j,k) = Tuple(😄)
        A[i,j,k] = @fastmath A[i,j,k] .+ Δt .* (Ȧ[i,j,k] .+ 0.5Δt .* F[i,j,k])
    end
    return nothing
end


function Lvelupdate_3D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j,k) = Tuple(😄)
        Ȧ[i,j,k] = @fastmath Ȧ[i,j,k] .+ 0.5Δt .* (F[i,j,k] .+ M[i,j,k] - ηₓ^n * η^2 * sin(A[i,j,k]) - 2/η * Ȧ[i,j,k])
    end
    return nothing
end


function Lupdate_3D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s)

    #ηₓ,η = Lηtime(time,fₐ);
    n = 1

    ηᵪ = [2.8 3 3.2 3.4 3.6 0 6 1]
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_3D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_3D!(A,Δt,Ȧ,F)

    LLaplacian_3D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_3D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_3D! (generic function with 1 method)

In [29]:
function Srun_3D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats,cont,low,B,aang,poggers,cap,pic)


    

    M₁ = zeros(N,N,N);
    M₂ = zeros(N,N,N);

    F₁ = zeros(N,N,N);
    F₂ = zeros(N,N,N);

    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N,N);
    angle = zeros(N,N,N);
    angler!(angle,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = cap

    tracker = []
    cores = []
    thr = 1
    lo = low
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    axionmax = []
    axionmin = []
    
    mappos = cgrad([RGBA(0, 1, 0, 0), RGBA(1,0,0,0.5), RGBA(1, 1, 1, 1)])
    
    Ȧ = zeros(N,N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N,N);

    F = zeros(N,N,N);

    LLaplacian_3D!(M,angle,Δx)

    tim = t₁
    
    ay,az = minmax(angll)
    
    
    if aang != 0
        Ȧ = B
        strng = 0
        mng = 2
        angll = aang
        ay = 0
        az = 0
    end
    
    for 🥪 ∈ 1+cont:repeats+cont
        
        

        while tim < 0.5*🥪 + t₁
        #while strng != 0
            
            if az ≥ 1 && ay ≤ -1

                tim = round(tim,digits = 7);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angle,A₁,A₂);
                    
                    

                    strng,loc = Cores_3D!(N,angle,thr)
                    
                    loc₁ = []
                    loc₂ = []
                    loc₃ = []

                    for i ∈ 1:length(loc)
                        append!(loc₁,loc[i][1])
                        append!(loc₂,loc[i][2])
                        append!(loc₃,loc[i][3])
                    end
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = 4/3 * π* Abin.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = 4/3 * π* Abins.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    
                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)
                    ρ = Ȧ.^2
                    
                    rho = mean(ρ)
                    
                    lag = log.(ρ/rho)
                    
                    amax = maximum(lag)
                    amin = minimum(lag)
                    
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
                    
#                     PyPlot.imsave("3D/QCDTOWN/String/Den/"*string(i)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)), vmin = 0,vmax = 6,cmap = "inferno")



                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))
                    
                    

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = 4/3 * π* Abint.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                    append!(axenergy,[Abint])



                    

                    ploot = Plots.plot(loc₁,loc₂,loc₃,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false,title = "η = "*string(tim))
                    Plots.savefig(ploot,"3D/QCDTOWN/String/"*string(i)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png")


                    ay,az = minmax(angll)
                    
                    
                    
                    setting_3D!(lag)


#                     vale = findall(x->x>5, lag)
#                     #mood = Tuple.(findall(>(5),log.(ρ/rho)));



#                     cmap = Plots.ColorGradient(:inferno)
#                     cmap.colors[1] = RGBA(0.,0.0,0.0,0.0)
                    
                    
                    ploot = Plots.plot(lag,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false,title = "η = "*string(tim),cmap = mappos)
                    Plots.savefig(ploot,"3D/QCDTOWN/String/O/"*string(i)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png")


                end
                if tim < 3
                
                    EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(3/tim)^3.34,tim,fₐ,r,s)
                    tim = tim + Δt*(3/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_3D!(M,angll,Δx)
                elseif mng == 2
                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_3D!(M,angll,Δx)
                end



                tim = round(tim,digits = 7);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angler!(angle,A₁,A₂);

                    strng,loc = Cores_3D!(N,angle,thr)

                    

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                    # f_image = FFTW.fft(moo)
                    # f_images = (abs.(f_image)).^2
                    # f_images = collect(Iterators.flatten(f_images))

                    # Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    # Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    #append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = 4/3 * π* Abins.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    ρ = Ȧ.^2
                    rho = mean(ρ)
                    
                    lag = log.(ρ/rho)
                    
                    amax = maximum(lag)
                    amin = minimum(lag)
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = 4/3* π* Abint.* (kbins[2:end].^3 - kbins[1:end-1].^3)

                    append!(axenergy,[Abint])
                    
                    
                    setting_3D!(lag)


#                     vale = findall(x->x>5, lag)
#                     #mood = Tuple.(findall(>(5),log.(ρ/rho)));



#                     cmap = Plots.ColorGradient(:inferno)
#                     cmap.colors[1] = RGBA(0.,0.0,0.0,0.0)
                    
                    
                    ploot = Plots.plot(lag,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false,title = "η = "*string(tim),cmap = mappos)
                    Plots.savefig(ploot,"3D/QCDTOWN/String/O/"*string(i)*"/"*lpad( string(trunc(Int, pic + lo/x )) ,3,"0")*".png")

                    


                end
                
                if tim < 3
                    Lupdate_3D!(angle,Ȧ,M,F,Δx,Δt,tim,fₐ,s)
                    tim = tim + Δt
                else
                    Lupdate_3D!(angle,Ȧ,M,F,Δx,Δt*(3/tim)^3.34,tim,fₐ,s)
                    tim = tim + Δt*(3/tim)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("3D/QCDTOWN/String/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*string(r)*string(s)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"Angle",angle,"AngVel",Ȧ)



        save("3D/QCDTOWN/String/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*string(r)*string(s)*".jld","time",tracker,"number",cores,"lo",lo)
        save("3D/QCDTOWN/String/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*string(r)*string(s)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



 

        
    end


    

    return nothing
end

Srun_3D! (generic function with 1 method)

In [45]:
@time begin

#check whether or not boundary conditions are periodic or not
    
n = 6
N = 2^n;
M = Int(N)
    
println(N)

rₐ = 15
fₐ = 10 .^rₐ;

    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]

🎱 = [5e1 5e1 5e1 5e1 5e1 0]
🏠 = [1e-2 1e-1 1e0 1e1 1e2 0]


for i ∈ 7:7
        
    cont = 0
    low = 0
    Ȧ = 0#zeros(2N,2N)

    cap = 25
    pic = 0
    poggers = 1

    #print(string(i)*" ")


    Ȧ₁ = zeros(N,N,N);
    Ȧ₂ = zeros(N,N,N);


    #🪂,σ = 1, 🎱[i];
    C₁ = zeros(N,N,N)
    C₂ = zeros(N,N,N)
    angll = 0
    #Make_Oscillon!(angll,🪂,N,σ)
    #print(angll)
        
    rstr1 = zeros(M,M,M)
    istr1 = zeros(M,M,M)
#     rstr2 = zeros(M,M)
#     istr2 = zeros(M,M)
#     rstr3 = zeros(M,M)
#     istr3 = zeros(M,M)
#     rstr4 = zeros(M,M)
#     istr4 = zeros(M,M)
    
    Σ = 1
    Anti_string!(rstr1,istr1,M,Σ,N/2,N/2,N/2)
#     Clock_string!(rstr2,istr2,M,Σ, N/2,N/2)
        
        
#     Anti_2string!(rstr1,istr1,M,Σ)
#     Clock_2string!(rstr2,istr2,M,Σ)
        
#     Anti_string!(rstr3,istr3,M,Σ, 3N/4,N/2)
#     Clock_string!(rstr4,istr4,M,Σ, N/2,N/2)

#     rstr3 = rstr3[end:-1:1,:]
#     istr3 = istr3[end:-1:1,:]
#     rstr4 = rstr4[end:-1:1,:]
#     istr4 = istr4[end:-1:1,:]

    slot!(C₁,rstr1,Int(N/2),Int(N/2),Int(N/2))
    slot!(C₂,istr1,Int(N/2),Int(N/2),Int(N/2))
        
#     slot!(C₁,rstr2,Int(N/2),Int(3N/2))
#     slot!(C₂,istr2,Int(N/2),Int(3N/2))
        
#     slot!(C₁,rstr3,Int(3N/2),Int(N/2))
#     slot!(C₂,istr3,Int(3N/2),Int(N/2))
        
#     slot!(C₁,rstr4,Int(3N/2),Int(3N/2))
#     slot!(C₂,istr4,Int(3N/2),Int(3N/2))
        


    s = 6
    r = 6

    


    t₁ = 0.1

    LQCD = LPQ/(2*100)*0.4

    Δx = round(LQCD[5]/N,digits = 6)
    Δt = 0.001



        
    repeat = 5
        
        
#     Srun_3D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat,cont,low,Ȧ,angll,poggers,cap,pic)


end
    
    

end

64


LoadError: BoundsError: attempt to access 62×62×62 Array{Float64, 3} at index [1, 1, 63]